In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
%cd /content/drive/MyDrive/tensorflow
# !git clone https://github.com/tensorflow/models.git

/content/drive/MyDrive/tensorflow


In [ ]:
%cd /content/drive/MyDrive/tensorflow/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/MyDrive/tensorflow/models/research


In [ ]:
!pip install pip --upgrade

     |████████████████████████████████| 2.1 MB 5.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!cp object_detection/packages/tf2/setup.py . 
!python -m pip install .

Processing /content/drive/MyDrive/tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=20151364 sha256=47b8f2d6a3b36cf53a7105a29e86b8526db0ebde3fa73d037075df0f5537d28b
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ywslz9e/wheels/4e/3e/65/b6217737e169edb7672931685d90db29cd012b280c4fd3b5ea
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
!python object_detection/builders/model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

2022-04-10 20:49:07.663352: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 4.95s
I0410 20:49:12.612443 140336152233856 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 4.95s
[       OK ] ModelBuilderTF2Test.test_create_center_net_model
[ RUN      ] ModelBuilderTF2Test.test_create_experimental_model
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_experimental_model): 0.0s
I0410 20:49:12.614044 140336152233856 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_experimental_model): 0.0s
[       OK ] ModelBuilderTF2Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF2Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
INFO:te

In [ ]:
%cd /content/drive/MyDrive/tensorflow/scripts/preproccesing/
!python generate_tfrecord.py -x '/content/drive/MyDrive/tensorflow/workspace/training_demo/images/train' -l '/content/drive/MyDrive/tensorflow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/drive/MyDrive/tensorflow/workspace/training_demo/annotations/train.record'
!python generate_tfrecord.py -x '/content/drive/MyDrive/tensorflow/workspace/training_demo/images/test' -l '/content/drive/MyDrive/tensorflow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/drive/MyDrive/tensorflow/workspace/training_demo/annotations/test.record'


/content/drive/MyDrive/tensorflow/scripts/preproccesing
2022-04-10 20:52:54.654506: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: /content/drive/MyDrive/tensorflow/workspace/training_demo/annotations/train.record
2022-04-10 20:53:04.944753: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: /content/drive/MyDrive/tensorflow/workspace/training_demo/annotations/test.record


In [ ]:
%cd /content/drive/MyDrive/tensorflow/workspace/training_demo
#%load_ext tensorboard
#%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

/content/drive/MyDrive/tensorflow/workspace/training_demo


In [ ]:
!python model_main_tf2.py --model_dir= models/ssd_mobilenet_v2 --pipeline_config_path=models/ssd_mobilenet_v2/pipeline.config

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir models/ssd_mobilenet_v2/checkpoint/ --output_directory ./exported-models/my_model 

In [ ]:
import os

from tensorflow.keras.preprocessing import image

pb_model_dir = "/content/drive/MyDrive/tensorflow/workspace/training_demo/models/ssd_mobilenet_v2/saved_model"
h5_model = "/content/drive/MyDrive/tensorflow/workspace/training_demo/exported-models/model.h5"

# Loading the Tensorflow Saved Model (PB)
model = tf.saved_model.load(pb_model_dir)
#print(model.summary())

# Saving the Model in H5 Format
tf.keras.models.save_model(model, h5_model)

# Loading the H5 Saved Model
loaded_model_from_h5 = tf.keras.models.load_model(h5_model)
print(loaded_model_from_h5.summary())

AttributeError: ignored

In [ ]:
import tensorflow as tf

path = "/content/drive/MyDrive/tensorflow/workspace/training_demo/exported-models/my_model/saved_model"

def create_model():
    baseModel =  tf.keras.models.load_model(path)
    baseModel.summary()   
    model_output = tf.keras.layers.Dense(3, name="head_supervised_new")(baseModel)
    model = tf.keras.Model(inputs=baseModel.input, outputs=model_output)
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])  
    return model
model = create_model()

ValueError: ignored